## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [28]:
#скачаем библиотеки
import pandas as pd
from IPython.display import HTML, display
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [29]:
#посмотрим на датафрейм
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2. Предобработка данных

### Обработка пропусков

In [30]:
#посмотрим в каких столбцах  датафрейма есть пропуски
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [31]:
#Пропуски имеются в двух столбцах - days_employed' и 'total_income'
#Сведем все строки с пропусками в отдельный датфрейм
data_NaN=data[(data['days_employed'].isnull())|(data['total_income'].isnull())]
display(data_NaN.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [32]:
data_NaN.shape#посмотрим на размер датафрейма data_NaN.

(2174, 12)

In [33]:
#Строки с пропусками в'days_employed'  совпали с пропусками в 'total_income' в 100% случаях и равно 2174 (около 10%)
#Проверим версию что пропуски относятся  к определенному типу дохода.
data_NaN['income_type'].value_counts(normalize=True)

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

In [34]:
#Посмотрим ка распределяется займы по типу дохода по всему датафрейму
data['income_type'].value_counts(normalize=True)

сотрудник          0.516562
компаньон          0.236237
пенсионер          0.179141
госслужащий        0.067782
безработный        0.000093
предприниматель    0.000093
студент            0.000046
в декрете          0.000046
Name: income_type, dtype: float64

In [35]:
#Анализ  показывает что распредление займов по типу дохода с пропусками  и без совпали. 
#Что означает что нет причины пропусков связанной с типом доходов. 
#Таким образом мы можем расчитать средние значения по каждому типу занятости и заменить ими пропуски. 
#Сделаем расчеты средних значений и проведем замену пропусков  через цикл. 
income_type=['сотрудник', 'компаньон','пенсионер','госслужащий','предприниматель']
for row in income_type:
    income_mean = data.loc[data.loc[:,'income_type']==row]['total_income'].mean()
    print('Среднее значение дохода для типа занятости: ',row,' ',income_mean)
    data.loc[data['income_type'] == row, 'total_income'] = data.loc[data['income_type'] == row, 'total_income'].fillna(income_mean)

Среднее значение дохода для типа занятости:  сотрудник   161380.26048788553
Среднее значение дохода для типа занятости:  компаньон   202417.4614617771
Среднее значение дохода для типа занятости:  пенсионер   137127.4656901654
Среднее значение дохода для типа занятости:  госслужащий   170898.30992266268
Среднее значение дохода для типа занятости:  предприниматель   499163.1449470857


   Мы обнаружили пропуски в двух столбцах: 'days_employed' и 'total_income'. Так же видно, что пропуски в обоих столбцах совпали в 100% случаях. Данные по трудовому стажу битые. Соответственно мы не можем ориентироваться на них. Проанализировали распределение доходов по типу занятости в общем датафрейме и в датафремйме с пропусками, мы обнаружили что они совпадают. На основании чего можно утверждать, что пропуски не связаны с типом дохода. Поэтому пропуски можно заменить на среднее значение. Мы отсортировали доходы и для каждого типа занятости посчитали среднее значение, в последствии заменив им пропуски

### Замена типа данных

In [36]:
#в датафрейме имеется два столбца с вещественным типом данных - 'days_employed' и 'total_income'
#переведем вещественный тип в целочисленный. Для столбца 'days_employed' заменим пропуски нулем и после переведем данные в тип int
data['days_employed'] = data['days_employed'].fillna(0)
data['days_employed'] = data['days_employed'].astype(int)


In [37]:
data['total_income']=data['total_income'].astype(int)#тем же методом переведем столбец 'total_income' в целочисленный тип. 

In [38]:
data.info() #просмотрим результат

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

  В датафрейме имелись два столбца с вещественным типом данных. Для наших целей достаточным является целочисленный, для перевода подходит метод astype(int).

### Обработка дубликатов

In [39]:
#посмотрим на наш датафрейм 
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [40]:
#в столбце 'education'есть данные отличающиеся регистром. Напрнимер 'СРЕДНЕЕ' и 'среднее'. Приведем данные к единому регистру 
data['education'] = data['education'].str.lower()

In [41]:
#посчитаем и удалим полные дубликаты. 
print('Всего в датафрейме дубликатов: ',data.duplicated().sum())  
data=data.drop_duplicates().reset_index(drop=True) 

Всего в датафрейме дубликатов:  71


В датафрейме нет уникального id заемщика, которое строго гарантировало принадлежность данных к одному и тому же заемщику. Но полное совпадение всех параметров маловероятно, поэтому удаляем дубликаты.  

### Лемматизация

In [42]:
#просмотрим  уникальные значения цели кредита и их количество
print(data['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [43]:
#В столбце 'purpose' одни и те же цели определены разными терминами. Например'получение высшего образования' и 'высшее образование' 
#Напишем функию "purpose_lemmatize"  которая получает строку и возвращает леммы. Добавим стобец с леммами в датафрейм
def purpose_lemmatize(row):
    return m.lemmatize(row['purpose'])
data['purpose_lemmas'] = data.apply(purpose_lemmatize, axis=1)
data['purpose_lemmas']

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21449                  [операция,  , с,  , жилье, \n]
21450               [сделка,  , с,  , автомобиль, \n]
21451                              [недвижимость, \n]
21452    [на,  , покупка,  , свой,  , автомобиль, \n]
21453             [на,  , покупка,  , автомобиль, \n]
Name: purpose_lemmas, Length: 21454, dtype: object

In [44]:
#найдем ключи для отбора категорий. 
counter = Counter()
for row in data['purpose_lemmas']:
    counter.update(row)
print(counter)


Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


In [45]:
#Получились следующие цели кредита:
#'ипотека' ключи: 'недвижимость','жилье'
#'образование' ключи: 'образование'
#'потребительский' ключи: 'свадьба'
#'автомобиль' ключи: 'автомобиль'

In [46]:
#напишем функцию которая получает строку с леммами, ищет ключ и возвращает цель. Добавим в датафрейм столбец 'purpose_category' 
def purpose_category(row):
    lemmas = row['purpose_lemmas']
    if 'свадьба' in lemmas:
        return 'потребительский'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'недвижимость' in lemmas or 'жилье'  in lemmas:
        return 'ипотека'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    else:
        return 'другое'
data['purpose_category'] = data.apply(purpose_category, axis=1)
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",ипотека
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",ипотека
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",потребительский
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",ипотека
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",ипотека
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",потребительский
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",ипотека


### Вывод

В результате лематизации сгруппировали цели кредита в 4 категории: 'потребительский', автомобиль', 'ипотека', 'образование'. На основании которых, мы сможем ответить на вопрос "Как разные цели кредита влияют на его возврат в срок?"

### Категоризация данных

In [47]:
#Найдем группы распределения дохода для этого разобьем столбец 'total_income' на квантили (низкий, ниже среднего, средний, выше среднегою,высокий)
#разобьем уровень дохода на квантили:'низкий', 'ниже среднего','средний','выше среднего','высокий'
pd.qcut(data['total_income'], 5)

0        (214618.2, 2265604.0]
1          (98537.6, 135448.4]
2         (135448.4, 162638.8]
3        (214618.2, 2265604.0]
4         (135448.4, 162638.8]
                 ...          
21449    (214618.2, 2265604.0]
21450     (135448.4, 162638.8]
21451     (20666.999, 98537.6]
21452    (214618.2, 2265604.0]
21453     (20666.999, 98537.6]
Name: total_income, Length: 21454, dtype: category
Categories (5, interval[float64]): [(20666.999, 98537.6] < (98537.6, 135448.4] < (135448.4, 162638.8] < (162638.8, 214618.2] < (214618.2, 2265604.0]]

In [48]:
#Проведем категоризацию заемщиков по уровню дохода
def total_income_category(total_income):
        """
        Возвращает категорию  группы по значению месячного дохода, используя правила:
        
        - 'низкий', при значение total_income <=98537
        - 'ниже среднего',  при значении  98537  < total_income <= 135448
        - 'средний', при значениии 135448  < total_income <= 162638
        - 'выше среднего' при значении 162638 < total_income <= 214618
        - 'высокий', во всех остальных случаях.
        
        """
        if total_income <=98537: return 'низкий'
        if total_income >98537 and total_income <=135448: return 'ниже среднего'
        if total_income >135448 and total_income <=162638: return 'средний'
        if total_income >162638 and total_income <=214618: return 'выше среднего'
        else: return 'высокий'
        
data['total_income_category']=data['total_income'].apply(total_income_category)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_category,total_income_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",ипотека,высокий
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,ниже среднего
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",ипотека,средний
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",потребительский,средний


In [49]:
#посмотрим как распределяется заемщики по количеству детей 
data['children'].value_counts(normalize=True)

 0     0.656801
 1     0.224107
 2     0.095646
 3     0.015382
 20    0.003542
-1     0.002191
 4     0.001911
 5     0.000420
Name: children, dtype: float64

In [50]:
#Разобьем на четыре группы: 'нет детей','один ребенок','два ребенка','многодетные'
def children_category(children):
        """
        Возвращает категорию  группы в зависимости от количества детей, используя правила:
        
        - 'нет детей', при значение children <=0
        - 'один ребенок',  при значении children ==1
        - 'два ребенка', при значениии children ==2
        - 'многодетные', во всех остальных случаях.
        
        """
        if children  <=0: return 'нет детей'
        if children ==1: return 'один ребенок'
        if children ==2: return 'два ребенка'
        else: return 'многодетные'
        
data['children_category']=data['children'].apply(children_category)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_category,total_income_category,children_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",ипотека,высокий,один ребенок
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,ниже среднего,один ребенок
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",ипотека,средний,нет детей
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",потребительский,средний,нет детей


### Вывод

Проведена категоризация по уровню дохода, что поможет ответить на вопрос как влияет уровень дохода на просроченную задолженность. Выделили 4  категории по наличию детей, что в свою очередь покажет как наличие детей влияет на задолженность. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [51]:
data_children = data.groupby('children_category').agg({'debt': ['sum', 'count']})
data_children['children_conversion']=data_children['debt']['sum']/data_children['debt']['count']*100
display(data_children.sort_values(by='children_conversion', ascending=False))


debt        children_conversion
                    sum  count                    
children_category                                 
два ребенка         194   2052            9.454191
один ребенок        444   4808            9.234609
многодетные          39    456            8.552632
нет детей          1064  14138            7.525817

### Вывод

  Чаще всего просроченная задолженность по кредиту появляется у людей, имеющих двух детей - в 9,45% случаях, одного ребенка - в 9,23 % случаях. Менее всего допускают "просрочку" клиенты без детей. Многодетные очень малочисленная группа заемщиков - их всего около 2 %. При этом они более дисциплинированные чем клиенты с одним или двумя детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [52]:
data_family = data.groupby('family_status').agg({'debt': ['sum', 'count']})
data_family['family_conversion']=data_family['debt']['sum']/data_family['debt']['count']*100
display(data_family.sort_values(by='family_conversion', ascending=False))


debt        family_conversion
                       sum  count                  
family_status                                      
Не женат / не замужем  274   2810          9.750890
гражданский брак       388   4151          9.347145
женат / замужем        931  12339          7.545182
в разводе               85   1195          7.112971
вдовец / вдова          63    959          6.569343

### Вывод

   Люди не в браке самые недисциплинированные заемщики. Они допускают просроченную задолженность в 9,75% случаях. Гражданский брак немногим лучше влияет на возврат долгов - просрочка в 9,35% случаях. Официальный брак дисциплинирует лучше людей, просрочка меньше -  7,55% .  На удивление люди, находящиеся в разводе и потерявшие партнера самые дисциплинированные заемщики, просрочка в 7,11% и 6,57% соответственно. Но надо отметить что их группа малочисленна их примерно поровну около 4-5 %

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [53]:
data_income = data.groupby('total_income_category').agg({'debt': ['sum', 'count']})
data_income['income_conversion']=data_income['debt']['sum']/data_income['debt']['count']*100
display(data_income.sort_values(by='income_conversion', ascending=False))


debt       income_conversion
                       sum count                  
total_income_category                             
средний                384  4290          8.951049
выше среднего          358  4291          8.343044
ниже среднего          355  4291          8.273130
низкий                 344  4291          8.016779
высокий                300  4291          6.991377

### Вывод

  Уровень дохода не оказывает сильное влияние на возврат задолжжености. Различные группы допускают просрочку  от 8,0 до 8,95 %. Немного лучше платят люди  с высоким доходом - около 7% просроченных платежей 

- Как разные цели кредита влияют на его возврат в срок?

In [54]:
data_purpose = data.groupby('purpose_category').agg({'debt': ['sum', 'count']})
data_purpose['purpose_conversion']=data_purpose['debt']['sum']/data_purpose['debt']['count']*100
display(data_purpose.sort_values(by='purpose_conversion', ascending=False))


debt        purpose_conversion
                  sum  count                   
purpose_category                               
автомобиль        403   4306           9.359034
образование       370   4013           9.220035
потребительский   186   2324           8.003442
ипотека           782  10811           7.233373

  Наиболее дисциплинированные клиенты, взявшие ипотеку, они допускают просрочку в 7,23% случаях. Хуже всего платят по автокредитам - уровень просрочки 9,35% и почти столько же на образование 9,22%.  Просроченные платежи по потребительскому кредиту около 8,0 %.

### Шаг 4. Общий вывод

В процессе исследования данных были выделены группы заемщиков по четырем признакам. В первой группе заёмщик распределены по признаку наличия детей, во второй по семейному положению, в третей по уровню дохода и в четвертой по целям кредита. По каждому признаку определено его влияние на просроченные платежи. Наибольшее влияние на оплату оказывает семейное положение, так заемщики вне брака, допускают больше всего просрочек. Менее всего, что удивительно, на своевременную оплату оказывает уровень дохода. Разброс колеблется в пределах одного процента в зависимости от уровня дохода. Но немногим лучше платят клиенты с самыми высокими доходами. 